<h2>Groupby Practice</h2>

In [1]:
import numpy as np
import pandas as pd

In [8]:
vet_data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'], 
        'age': [2.5, 3, 0.5, 6, 5, 2, 4.5, 4, 7, 3], 
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1], 
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = list('abcdefghij')

df = pd.DataFrame(vet_data, index=labels)
df

,animal,age,visits,priority
a,cat,2.5,1,yes
b,cat,3.0,3,yes
c,snake,0.5,2,no
d,dog,6.0,3,yes
e,dog,5.0,2,no
f,cat,2.0,3,no
g,snake,4.5,1,no
h,cat,4.0,1,yes
i,dog,7.0,2,no
j,dog,3.0,1,no


<h3>pd.Sereis.value_counts() displays value counts for one column data</h3>

In [9]:
df.animal.value_counts()

dog      4
cat      4
snake    2
Name: animal, dtype: int64

In [4]:
df.describe()

,age,visits
count,10.00000,10.000000
mean,3.75000,1.900000
std,1.94722,0.875595
min,0.50000,1.000000
25%,2.62500,1.000000
50%,3.50000,2.000000
75%,4.87500,2.750000
max,7.00000,3.000000


In [5]:
df.animal.describe()

count      10
unique      3
top       dog
freq        4
Name: animal, dtype: object

<h3>Q: What's the average age of dogs, cats and snakes? </h3>
<h3>Create a new DataFrame that shows the average age for each group of animals?</h3>

<h3>Approach 1: use boolean array</h3>

In [11]:
animal = df.animal == 'dog'
df[animal]

,animal,age,visits,priority
d,dog,6.0,3,yes
e,dog,5.0,2,no
i,dog,7.0,2,no
j,dog,3.0,1,no


In [12]:
df[animal].age.mean()

5.25

In [13]:
age ={}

for a in ('dog','cat','snake'):
    m = df[df.animal == a].age.mean(0)
    age[a] = m

age_df = pd.Series(age)
age_df

dog      5.250
cat      2.875
snake    2.500
dtype: float64

<h3>Approach 2: use groupby</h3>

In [14]:
animal = df.groupby('animal')
type (animal)

pandas.core.groupby.generic.DataFrameGroupBy

In [15]:
# groupby.ngroups: returns the number of groups
animal.ngroups

3

In [16]:
# groupby.size(): returns number of rows in each group
# It produces the same result as in df.animal.value_counts()
animal.size()

animal
cat      4
dog      4
snake    2
dtype: int64

In [17]:
# groupby.groups: returns a dict where index=group name, value=[row indices in that group] 
animal.groups

{'cat': ['a', 'b', 'f', 'h'], 'dog': ['d', 'e', 'i', 'j'], 'snake': ['c', 'g']}

In [18]:
# groupby.get_group(name): returns the values in that group (works the same as dict)
animal.get_group('dog')

,animal,age,visits,priority
d,dog,6.0,3,yes
e,dog,5.0,2,no
i,dog,7.0,2,no
j,dog,3.0,1,no


In [19]:
# use pandas mean() method to find the average age

animal.get_group('dog').age.mean()

5.25

In [20]:
# You can also conveniently do this to get the average age for each group
# Notice the subtle difference in the two different approach

# Shorthand notation: animal.age is a SeriesGroupBy object
animal.age.mean()

animal
cat      2.875
dog      5.250
snake    2.500
Name: age, dtype: float64

In [21]:
# animal[['age']] is a DataFrameGroupBy object
animal[['age']].mean()

,age
animal,
cat,2.875
dog,5.250
snake,2.500


In [22]:
# Since animal.age is a Series Object, we can use describe() method

print (animal.age)
animal.age.describe()

,count,mean,std,min,25%,50%,75%,max
animal,,,,,,,,
cat,4.0,2.875,0.853913,2.0,2.375,2.75,3.25,4.0
dog,4.0,5.250,1.707825,3.0,4.500,5.50,6.25,7.0
snake,2.0,2.500,2.828427,0.5,1.500,2.50,3.50,4.5


In [23]:
# You can use pandas sum() method to add up ages in the group

animal.age.sum()

animal
cat      11.5
dog      21.0
snake     5.0
Name: age, dtype: float64

<h3>Use agg() for multiple aggregation operations</h3>

In [24]:
animal.age.agg([np.mean, np.std, np.max, np.min])

,mean,std,amax,amin
animal,,,,
cat,2.875,0.853913,4.0,2.0
dog,5.250,1.707825,7.0,3.0
snake,2.500,2.828427,4.5,0.5


In [25]:
# Equivalent way to perform the same task

animal.age.agg(['mean', 'std', 'max', 'min'])

,mean,std,max,min
animal,,,,
cat,2.875,0.853913,4.0,2.0
dog,5.250,1.707825,7.0,3.0
snake,2.500,2.828427,4.5,0.5


<h3>Use Python apply(func) function to the GroupBy object</h3>

In [27]:
norm = lambda x: (x - x.mean())/x.std()

animal[['age']].apply(norm)

,age
a,-0.439155
b,0.146385
c,-0.707107
d,0.439155
e,-0.146385
f,-1.024695
g,0.707107
h,1.317465
i,1.024695
j,-1.317465


<h3>Use agg() to apply different functions on differnt columns</h3>
<h3>Let's say we want to apply two functions on the 'age' column and another funtion on the 'visits' column</h3>

In [42]:
# Create a dictionary for functions

dict_fun = {'age':['max','min'], 'visits':['mean']}

In [43]:
animal.agg(dict_fun)

age      visits
        max  min   mean
animal                 
cat     4.0  2.0    2.0
dog     7.0  3.0    2.0
snake   4.5  0.5    1.5

<h3>You can also change the column labels after the aggregation</h3>

In [44]:
animal.agg(
        max_age = ('age','max'), 
        min_age = ('age', 'min'), 
        ave_visits = ('visits','mean')
        )

,max_age,min_age,ave_visits
animal,,,
cat,4.0,2.0,2.0
dog,7.0,3.0,2.0
snake,4.5,0.5,1.5
